
2/12/24:
- Based on reviewer feedback, provide info on lowess fits that serve as the bases for Figure 3.
- The script here is to:
  - Include category info in PDF based on [this post](https://stackoverflow.com/questions/1180115/add-text-to-existing-pdf-using-python) and [this](https://stackoverflow.com/questions/71599203/how-to-add-text-annotation-to-existing-pdf-using-pdf-annotate-python).
  - Rename files to put order and category info.

In [38]:
import pandas as pd
from pathlib import Path
from pdf_annotate import PdfAnnotator, Appearance, Location
from tqdm import tqdm

proj_dir   = Path.home() / "projects/plant_sci_hist"
work_dir   = proj_dir / "4_topic_model/4_4_over_time/_lowess/_selected"

In [5]:
# Get all pdf files
pdfs = [f for f in list(work_dir.iterdir()) if str(f).find(".pdf") != -1]
len(pdfs)

87

In [41]:
# Get the ordering information from spreadsheet
df = pd.read_excel(work_dir / "ordering.xlsx")

df.head(2)

,Order,Topic,Peak,1st_reach_thr,Trend,Stable,Category
0,1,58,0,0,-1,1,Stable
1,2,56,3,0,-1,1,Stable


In [43]:
# Modify and rename pdf
order = df["Order"].tolist()
tocs  = df["Topic"].tolist()
cats  = df["Category"].tolist()

for pdf in tqdm(pdfs):
  # Get topic, order, and category info
  toc   = int(str(pdf).split("_topic_")[-1].split("_lowess_")[0])
  order = df[df["Topic"] == toc]["Order"].values[0]
  cat   = df[df["Topic"] == toc]["Category"].values[0]

  # Add annotation
  annotator = PdfAnnotator(str(pdf))
  annotator.add_annotation(
    'text',
    Location(x1=50, y1=500, x2=200, y2=100, page=0),
    Appearance(content=f'Category={cat}, order={order}', fill=(0,0,0)), 
    )
  
  # Generate output
  pdf_out = work_dir / f"{order}_{cat}_topc_{toc}.pdf"
  annotator.write(str(pdf_out))


100%|██████████| 87/87 [00:01<00:00, 52.31it/s]


## ___Testing___

In [31]:
from pdf_annotate import PdfAnnotator, Appearance, Location

# This is not very helpful, particularly, need more info on appearance
help(Appearance)

Help on class Appearance in module pdf_annotate.config.appearance:

class Appearance(builtins.object)
 |  Appearance(stroke_color=(0, 0, 0), stroke_width=1, border_style='S', dash_array=None, line_cap=None, line_join=None, miter_limit=None, stroke_transparency=None, fill=None, fill_transparency=None, content='', font_size=12, text_align='left', text_baseline='middle', line_spacing=1.2, wrap_text=True, image=None, appearance_stream=None, xobjects=None, graphics_states=None, fonts=None) -> None
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Method generated by attrs for class Appearance.
 |  
 |  __ge__(self, other)
 |      Method generated by attrs for class Appearance.
 |  
 |  __gt__(self, other)
 |      Method generated by attrs for class Appearance.
 |  
 |  __init__(self, stroke_color=(0, 0, 0), stroke_width=1, border_style='S', dash_array=None, line_cap=None, line_join=None, miter_limit=None, stroke_transparency=None, fill=None, fill_transparency=None, content

In [36]:

test_pdf = str(work_dir / "test.pdf")

annotator = PdfAnnotator(test_pdf)

annotator.add_annotation(
  'text',
  Location(x1=50, y1=500, x2=200, y2=100, page=0),
  Appearance(content='Review...', fill=(0,0,0)), 
  )

test_out = str(work_dir / "test_annotated.pdf")

annotator.write(test_out)
